In [ ]:
%load_ext autoreload
%autoreload 2
%store -r the_page
#the_page

# 1. General Metadata of a Wikipedia Article

Let's say we are interested in the Wikipedia historical evolution of one article, the novel "The Camp of the Saints". We can get some data from Wikipedia itself, and some third party sources:

## 1.1 Basic Info from Wikipedia

In [ ]:
from ipywidgets import widgets
from IPython.display import display
from external.wikipedia import WikipediaDV, WikipediaAPI
wikipedia_dv = WikipediaDV(WikipediaAPI(domain='en.wikipedia.org'))

def on_button_clicked(b):
    global the_page
    if searchTerm.value == "":
        print(f"Please input a page name or id in the input field")
        return
    
    the_page = wikipedia_dv.get_page(searchTerm.value)
    
    if 'missing' in the_page:
        print(f"The page {searchTerm.value} was not found")
        
    display(the_page.to_frame('value'))

try:
    searchTerm = widgets.Text(the_page['title'], description='Page title:')
except:
    searchTerm = widgets.Text("The Camp of the Saints", description='Page title:')
button = widgets.Button(description="Search")

display(searchTerm,button)
button.on_click(on_button_clicked)
button.click()

## 1.2 General Stats from Xtools

In [ ]:
from external.xtools import XtoolsAPI, XtoolsDV
xtools_api = XtoolsAPI(project = 'en.wikipedia.org')
xtools_dv = XtoolsDV(xtools_api)
page_info = xtools_dv.get_page_info(the_page['title'])
page_info.to_frame('value')

## 1.3 Page Views from Wikimedia

In [ ]:
# Query request
from external.wikimedia import WikiMediaDV, WikiMediaAPI
wikimedia_api = WikiMediaAPI(project='en.wikipedia')
wikimedia_dv = WikiMediaDV(wikimedia_api)
views = wikimedia_dv.get_pageviews(the_page['title'], 'daily')

# Visualization
from visualization.listeners import DFListener
from ipywidgets import interact
listener = DFListener(views)
interact(listener.views_per_period, 
         begin=views.timestamp,
         end=views.timestamp.sort_values(ascending=False),
         granularity=['Yearly', 'Monthly', 'Weekly', 'Daily'])

# The df_plotted keeps a reference to the plotted data above
listener.df_plotted['views'].agg({
    'Total views': sum,
    'Max views period': max,
    'Min views period': min,
    'Average views': min,}).to_frame('Value')

In [ ]:
%store the_page

from IPython.display import HTML
HTML('<a href="2. Using WikiWho to analyze a page.ipynb" target="_blank">Go to next workbook</a>')